# Trainable Wavelet Fusion Model - Testing and Evaluation

This notebook tests the trained wavelet fusion model on sample CT-MRI image pairs and evaluates the fusion quality with comprehensive metrics and visualizations.

## 1. Import Required Libraries

In [1]:
import os
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import seaborn as sns
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

# PyWavelets and pytorch_wavelets for wavelet transforms
from pytorch_wavelets import DWTForward, DWTInverse

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Set matplotlib style
plt.style.use('default')
sns.set_palette("husl")

Using device: cuda


## 2. Load the Trained Model

Define the model architecture and load the trained weights.

In [2]:
# Load the trained model
model_path = 'checkpoints_wavelet/wavelet_fusion_best.pt'
if os.path.exists(model_path):
    model = WaveletFusionNet(wave='haar').to(device)
    
    # Load checkpoint which contains model, optimizer, and config
    checkpoint = torch.load(model_path, map_location=device)
    
    # Extract model state_dict from checkpoint
    if isinstance(checkpoint, dict) and 'model' in checkpoint:
        model.load_state_dict(checkpoint['model'])
        print(f"Model loaded successfully from {model_path}")
        
        # Display checkpoint info if available
        if 'cfg' in checkpoint:
            print(f"Training config: {checkpoint['cfg']}")
    else:
        # If checkpoint is just the state_dict directly
        model.load_state_dict(checkpoint)
        print(f"Model loaded successfully from {model_path}")
    
    model.eval()
    
    # Display learned parameters
    print(f"\nLearned Fusion Weights:")
    print(f"  Low-frequency (α_L):  {model.alpha_low.item():.4f}")
    print(f"  LH detail (α_LH):     {model.alpha_lh.item():.4f}")
    print(f"  HL detail (α_HL):     {model.alpha_hl.item():.4f}")
    print(f"  HH detail (α_HH):     {model.alpha_hh.item():.4f}")
else:
    print(f"Model checkpoint not found at {model_path}. Please train the model first.")
    model = None

Model checkpoint not found at checkpoints_wavelet/wavelet_fusion_best.pt. Please train the model first.


## 3. Load and Preprocess Test Images

Define helper functions to load and preprocess CT-MRI image pairs.

In [3]:
def load_image_pair(ct_path, mri_path, resize=None):
    """Load and preprocess a CT-MRI image pair."""
    ct = cv2.imread(ct_path, cv2.IMREAD_GRAYSCALE)
    mri = cv2.imread(mri_path, cv2.IMREAD_GRAYSCALE)
    
    if ct is None or mri is None:
        raise FileNotFoundError(f"Could not load images: {ct_path}, {mri_path}")
    
    if resize is not None:
        H, W = resize
        ct = cv2.resize(ct, (W, H), interpolation=cv2.INTER_AREA)
        mri = cv2.resize(mri, (W, H), interpolation=cv2.INTER_AREA)
    
    # Normalize to [0, 1]
    ct = ct.astype(np.float32)
    mri = mri.astype(np.float32)
    if ct.max() > 1.0: ct /= 255.0
    if mri.max() > 1.0: mri /= 255.0
    
    return ct, mri

def get_test_image_pairs(ct_dir, mri_dir, num_samples=5):
    """Get a list of test image pairs."""
    ct_files = sorted(glob.glob(os.path.join(ct_dir, '*.png')))
    mri_files = sorted(glob.glob(os.path.join(mri_dir, '*.png')))
    
    # Match files by basename
    pairs = []
    for ct_file in ct_files:
        ct_basename = os.path.basename(ct_file)
        mri_file = os.path.join(mri_dir, ct_basename)
        if os.path.exists(mri_file):
            pairs.append((ct_file, mri_file))
    
    # Select a subset for testing
    if len(pairs) > num_samples:
        # Select evenly spaced samples
        indices = np.linspace(0, len(pairs)-1, num_samples, dtype=int)
        pairs = [pairs[i] for i in indices]
    
    return pairs

# Get test image pairs
ct_test_dir = 'Harvard-Medical-Image-Fusion-Datasets/MyDatasets/CT-MRI/test/CT'
mri_test_dir = 'Harvard-Medical-Image-Fusion-Datasets/MyDatasets/CT-MRI/test/MRI'

test_pairs = get_test_image_pairs(ct_test_dir, mri_test_dir, num_samples=6)
print(f"Found {len(test_pairs)} test image pairs")

for i, (ct_path, mri_path) in enumerate(test_pairs[:3]):
    print(f"  Pair {i+1}: {os.path.basename(ct_path)}")

Found 6 test image pairs
  Pair 1: 16009.png
  Pair 2: 16013.png
  Pair 3: 2010.png


## 4. Perform Image Fusion

Define fusion methods including the trained model and baseline methods for comparison.

In [4]:
def fuse_with_model(model, ct, mri):
    """Fuse images using the trained model."""
    if model is None:
        return None
    
    with torch.no_grad():
        # Convert to tensors and add batch dimension
        ct_tensor = torch.from_numpy(ct)[None, None, ...].to(device)  # (1,1,H,W)
        mri_tensor = torch.from_numpy(mri)[None, None, ...].to(device)
        
        # Perform fusion
        fused_tensor = model(ct_tensor, mri_tensor)
        
        # Convert back to numpy
        fused = fused_tensor.squeeze().cpu().numpy()
        return np.clip(fused, 0, 1)

def fuse_average(ct, mri):
    """Simple average fusion."""
    return 0.5 * (ct + mri)

def fuse_max(ct, mri):
    """Max-based fusion."""
    return np.maximum(ct, mri)

def fuse_weighted(ct, mri, weight=0.6):
    """Weighted fusion favoring CT."""
    return weight * ct + (1 - weight) * mri

# Test fusion on the first image pair
if test_pairs and model is not None:
    ct_path, mri_path = test_pairs[0]
    ct_img, mri_img = load_image_pair(ct_path, mri_path)
    
    print(f"Testing fusion on: {os.path.basename(ct_path)}")
    print(f"Image shape: {ct_img.shape}")
    print(f"CT range: [{ct_img.min():.3f}, {ct_img.max():.3f}]")
    print(f"MRI range: [{mri_img.min():.3f}, {mri_img.max():.3f}]")
    
    # Test fusion
    fused_model = fuse_with_model(model, ct_img, mri_img)
    if fused_model is not None:
        print(f"Fused range: [{fused_model.min():.3f}, {fused_model.max():.3f}]")
        print("Fusion successful!")
    else:
        print("Model fusion failed")

## 5. Display Fusion Results

Visualize the fusion results for individual image pairs.

In [5]:
def display_fusion_results(ct, mri, fused_results, titles, figsize=(15, 10)):
    """Display original images and fusion results."""
    n_methods = len(fused_results)
    n_cols = min(4, n_methods + 2)  # +2 for original CT and MRI
    n_rows = (n_methods + 2 + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    # Flatten axes for easy indexing
    axes_flat = axes.flatten()
    
    # Display original images
    axes_flat[0].imshow(ct, cmap='gray')
    axes_flat[0].set_title('CT (Source A)', fontsize=12, fontweight='bold')
    axes_flat[0].axis('off')
    
    axes_flat[1].imshow(mri, cmap='gray')
    axes_flat[1].set_title('MRI (Source B)', fontsize=12, fontweight='bold')
    axes_flat[1].axis('off')
    
    # Display fusion results
    for i, (fused, title) in enumerate(zip(fused_results, titles)):
        if fused is not None:
            axes_flat[i + 2].imshow(fused, cmap='gray')
            axes_flat[i + 2].set_title(title, fontsize=12)
        else:
            axes_flat[i + 2].text(0.5, 0.5, 'N/A', ha='center', va='center', transform=axes_flat[i + 2].transAxes)
            axes_flat[i + 2].set_title(title, fontsize=12)
        axes_flat[i + 2].axis('off')
    
    # Hide unused subplots
    for i in range(n_methods + 2, len(axes_flat)):
        axes_flat[i].axis('off')
    
    plt.tight_layout()
    return fig

# Display results for the first test pair
if test_pairs and model is not None:
    ct_path, mri_path = test_pairs[0]
    ct_img, mri_img = load_image_pair(ct_path, mri_path)
    
    # Generate fusion results
    fused_model = fuse_with_model(model, ct_img, mri_img)
    fused_avg = fuse_average(ct_img, mri_img)
    fused_max = fuse_max(ct_img, mri_img)
    fused_weighted = fuse_weighted(ct_img, mri_img)
    
    fused_results = [fused_model, fused_avg, fused_max, fused_weighted]
    titles = ['Trained Model', 'Average', 'Maximum', 'Weighted (0.6)']
    
    fig = display_fusion_results(ct_img, mri_img, fused_results, titles)
    plt.suptitle(f'Fusion Results: {os.path.basename(ct_path)}', fontsize=16, fontweight='bold')
    plt.show()

## 6. Calculate Performance Metrics

Implement comprehensive metrics to evaluate fusion quality.

In [6]:
def calculate_ssim(img1, img2):
    """Calculate SSIM between two images."""
    from skimage.metrics import structural_similarity as ssim
    return ssim(img1, img2, data_range=1.0)

def calculate_psnr(img1, img2):
    """Calculate PSNR between two images."""
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    return 20 * np.log10(1.0 / np.sqrt(mse))

def calculate_gradient_magnitude(img):
    """Calculate gradient magnitude using Sobel operators."""
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    return np.sqrt(grad_x**2 + grad_y**2)

def calculate_edge_preservation(fused, ct, mri):
    """Calculate edge preservation metric."""
    # Calculate gradients
    grad_fused = calculate_gradient_magnitude(fused)
    grad_ct = calculate_gradient_magnitude(ct)
    grad_mri = calculate_gradient_magnitude(mri)
    
    # Maximum gradient from sources
    grad_max = np.maximum(grad_ct, grad_mri)
    
    # Edge preservation as correlation
    correlation = np.corrcoef(grad_fused.flatten(), grad_max.flatten())[0, 1]
    return correlation if not np.isnan(correlation) else 0.0

def calculate_entropy(img):
    """Calculate image entropy."""
    # Convert to 8-bit for histogram
    img_uint8 = (img * 255).astype(np.uint8)
    hist, _ = np.histogram(img_uint8, bins=256, range=(0, 256))
    hist = hist / hist.sum()  # Normalize
    hist = hist[hist > 0]  # Remove zeros
    return -np.sum(hist * np.log2(hist))

def calculate_mutual_information(img1, img2):
    """Calculate mutual information between two images."""
    # Convert to 8-bit
    img1_uint8 = (img1 * 255).astype(np.uint8)
    img2_uint8 = (img2 * 255).astype(np.uint8)
    
    # Joint histogram
    hist_2d, _, _ = np.histogram2d(img1_uint8.flatten(), img2_uint8.flatten(), bins=256)
    
    # Normalize
    hist_2d = hist_2d / hist_2d.sum()
    
    # Marginal histograms
    hist_1 = hist_2d.sum(axis=1)
    hist_2 = hist_2d.sum(axis=0)
    
    # Calculate MI
    mi = 0.0
    for i in range(256):
        for j in range(256):
            if hist_2d[i, j] > 0 and hist_1[i] > 0 and hist_2[j] > 0:
                mi += hist_2d[i, j] * np.log2(hist_2d[i, j] / (hist_1[i] * hist_2[j]))
    
    return mi

def evaluate_fusion_quality(fused, ct, mri):
    """Comprehensive evaluation of fusion quality."""
    if fused is None:
        return {}
    
    metrics = {}
    
    # SSIM with both source images
    metrics['SSIM_CT'] = calculate_ssim(fused, ct)
    metrics['SSIM_MRI'] = calculate_ssim(fused, mri)
    metrics['SSIM_Avg'] = (metrics['SSIM_CT'] + metrics['SSIM_MRI']) / 2
    
    # PSNR with both source images
    metrics['PSNR_CT'] = calculate_psnr(fused, ct)
    metrics['PSNR_MRI'] = calculate_psnr(fused, mri)
    metrics['PSNR_Avg'] = (metrics['PSNR_CT'] + metrics['PSNR_MRI']) / 2
    
    # Edge preservation
    metrics['Edge_Preservation'] = calculate_edge_preservation(fused, ct, mri)
    
    # Entropy (information content)
    metrics['Entropy'] = calculate_entropy(fused)
    
    # Mutual information with sources
    metrics['MI_CT'] = calculate_mutual_information(fused, ct)
    metrics['MI_MRI'] = calculate_mutual_information(fused, mri)
    metrics['MI_Avg'] = (metrics['MI_CT'] + metrics['MI_MRI']) / 2
    
    return metrics

# Test metrics calculation
if test_pairs and model is not None:
    ct_path, mri_path = test_pairs[0]
    ct_img, mri_img = load_image_pair(ct_path, mri_path)
    fused_model = fuse_with_model(model, ct_img, mri_img)
    
    metrics = evaluate_fusion_quality(fused_model, ct_img, mri_img)
    
    print("Sample Metrics for Trained Model:")
    for key, value in metrics.items():
        print(f"  {key}: {value:.4f}")

## 7. Compare Multiple Test Cases

Process multiple test image pairs and compare different fusion methods.

In [7]:
# Process all test pairs and collect metrics
results_data = []
fusion_methods = {
    'Trained Model': lambda ct, mri: fuse_with_model(model, ct, mri),
    'Average': fuse_average,
    'Maximum': fuse_max,
    'Weighted': fuse_weighted
}

if test_pairs and model is not None:
    print(f"Processing {len(test_pairs)} test pairs...")
    
    for i, (ct_path, mri_path) in enumerate(test_pairs):
        try:
            # Load image pair
            ct_img, mri_img = load_image_pair(ct_path, mri_path)
            image_name = os.path.basename(ct_path)
            
            print(f"  Processing {image_name}...")
            
            # Test each fusion method
            for method_name, method_func in fusion_methods.items():
                try:
                    fused = method_func(ct_img, mri_img)
                    metrics = evaluate_fusion_quality(fused, ct_img, mri_img)
                    
                    # Add metadata
                    metrics['Image'] = image_name
                    metrics['Method'] = method_name
                    metrics['Pair_Index'] = i + 1
                    
                    results_data.append(metrics)
                    
                except Exception as e:
                    print(f"    Error with {method_name}: {e}")
                    
        except Exception as e:
            print(f"  Error loading {ct_path}: {e}")
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results_data)
    print(f"\nCollected results for {len(results_df)} method-image combinations")
    print(f"Shape: {results_df.shape}")
    
    # Display sample results
    if not results_df.empty:
        print("\nSample Results:")
        display_cols = ['Image', 'Method', 'SSIM_Avg', 'PSNR_Avg', 'Edge_Preservation', 'Entropy']
        print(results_df[display_cols].head(8))
else:
    results_df = pd.DataFrame()
    print("No test pairs available or model not loaded.")

No test pairs available or model not loaded.


### Visualize Multiple Test Cases Side by Side

In [8]:
def display_multiple_cases(test_pairs, model, max_cases=4):
    """Display fusion results for multiple test cases."""
    n_cases = min(len(test_pairs), max_cases)
    
    fig, axes = plt.subplots(n_cases, 6, figsize=(18, 3*n_cases))
    if n_cases == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(n_cases):
        ct_path, mri_path = test_pairs[i]
        ct_img, mri_img = load_image_pair(ct_path, mri_path)
        
        # Generate fusions
        fused_model = fuse_with_model(model, ct_img, mri_img)
        fused_avg = fuse_average(ct_img, mri_img)
        fused_max = fuse_max(ct_img, mri_img)
        fused_weighted = fuse_weighted(ct_img, mri_img)
        
        # Display images
        images = [ct_img, mri_img, fused_model, fused_avg, fused_max, fused_weighted]
        titles = ['CT', 'MRI', 'Trained', 'Average', 'Maximum', 'Weighted']
        
        for j, (img, title) in enumerate(zip(images, titles)):
            if img is not None:
                axes[i, j].imshow(img, cmap='gray')
            else:
                axes[i, j].text(0.5, 0.5, 'N/A', ha='center', va='center')
            
            if i == 0:  # Add titles only to the first row
                axes[i, j].set_title(title, fontsize=12, fontweight='bold')
            axes[i, j].axis('off')
        
        # Add image name as row label
        axes[i, 0].text(-0.1, 0.5, os.path.basename(ct_path), 
                       transform=axes[i, 0].transAxes, rotation=90, 
                       ha='center', va='center', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    return fig

# Display multiple test cases
if test_pairs and model is not None and len(test_pairs) > 0:
    fig = display_multiple_cases(test_pairs, model, max_cases=min(4, len(test_pairs)))
    plt.suptitle('Fusion Results Comparison Across Multiple Cases', 
                fontsize=16, fontweight='bold', y=1.02)
    plt.show()

## 8. Visualize Metrics Summary

Create comprehensive visualizations and statistical summaries of the performance metrics.

In [9]:
if not results_df.empty:
    # Create summary statistics
    metric_columns = ['SSIM_Avg', 'PSNR_Avg', 'Edge_Preservation', 'Entropy', 'MI_Avg']
    summary_stats = results_df.groupby('Method')[metric_columns].agg(['mean', 'std', 'min', 'max'])
    
    print("Summary Statistics by Method:")
    print("=" * 80)
    for metric in metric_columns:
        print(f"\n{metric}:")
        print(summary_stats[metric].round(4))
    
    # Visualization 1: Bar plot comparing average metrics
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, metric in enumerate(metric_columns):
        if i < len(axes):
            mean_values = results_df.groupby('Method')[metric].mean()
            std_values = results_df.groupby('Method')[metric].std()
            
            bars = axes[i].bar(mean_values.index, mean_values.values, 
                              yerr=std_values.values, capsize=5, alpha=0.8)
            axes[i].set_title(f'{metric}', fontsize=12, fontweight='bold')
            axes[i].set_ylabel('Value')
            axes[i].tick_params(axis='x', rotation=45)
            axes[i].grid(True, alpha=0.3)
            
            # Highlight the best method
            best_idx = mean_values.argmax()
            bars[best_idx].set_color('red')
            bars[best_idx].set_alpha(1.0)
    
    # Hide the last subplot if we have an odd number of metrics
    if len(metric_columns) < len(axes):
        axes[-1].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Performance Metrics Comparison (Higher is Better)', 
                fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
else:
    print("No results data available for visualization.")

No results data available for visualization.


### Heatmap of Performance Metrics

In [10]:
if not results_df.empty:
    # Create a pivot table for heatmap
    pivot_data = results_df.pivot_table(index='Method', 
                                        columns='Image', 
                                        values='SSIM_Avg', 
                                        aggfunc='mean')
    
    # Create heatmap
    plt.figure(figsize=(14, 8))
    sns.heatmap(pivot_data, annot=True, cmap='YlOrRd', fmt='.3f', 
                cbar_kws={'label': 'Average SSIM'})
    plt.title('SSIM Performance Heatmap Across Test Images', 
              fontsize=16, fontweight='bold')
    plt.xlabel('Test Images')
    plt.ylabel('Fusion Methods')
    plt.tight_layout()
    plt.show()
    
    # Box plots for metric distributions
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    axes = axes.flatten()
    
    key_metrics = ['SSIM_Avg', 'PSNR_Avg', 'Edge_Preservation', 'Entropy']
    
    for i, metric in enumerate(key_metrics):
        sns.boxplot(data=results_df, x='Method', y=metric, ax=axes[i])
        axes[i].set_title(f'{metric} Distribution', fontweight='bold')
        axes[i].tick_params(axis='x', rotation=45)
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('Metric Distributions Across All Test Cases', 
                fontsize=16, fontweight='bold', y=1.02)
    plt.show()

### Ranking and Final Summary

In [11]:
if not results_df.empty:
    # Calculate overall ranking
    ranking_metrics = ['SSIM_Avg', 'PSNR_Avg', 'Edge_Preservation', 'Entropy', 'MI_Avg']
    method_scores = {}
    
    for method in results_df['Method'].unique():
        method_data = results_df[results_df['Method'] == method]
        
        # Calculate normalized scores (0-1) for each metric
        scores = []
        for metric in ranking_metrics:
            if metric in method_data.columns:
                # Normalize by the maximum value across all methods
                max_val = results_df[metric].max()
                min_val = results_df[metric].min()
                if max_val > min_val:
                    normalized = (method_data[metric].mean() - min_val) / (max_val - min_val)
                else:
                    normalized = 1.0
                scores.append(normalized)
        
        method_scores[method] = np.mean(scores)
    
    # Sort methods by overall score
    ranked_methods = sorted(method_scores.items(), key=lambda x: x[1], reverse=True)
    
    print("\n" + "="*60)
    print("FINAL RANKING - OVERALL PERFORMANCE")
    print("="*60)
    
    for i, (method, score) in enumerate(ranked_methods, 1):
        method_data = results_df[results_df['Method'] == method]
        print(f"\n{i}. {method} (Score: {score:.4f})")
        print(f"   • SSIM Average:        {method_data['SSIM_Avg'].mean():.4f} ± {method_data['SSIM_Avg'].std():.4f}")
        print(f"   • PSNR Average:        {method_data['PSNR_Avg'].mean():.4f} ± {method_data['PSNR_Avg'].std():.4f}")
        print(f"   • Edge Preservation:   {method_data['Edge_Preservation'].mean():.4f} ± {method_data['Edge_Preservation'].std():.4f}")
        print(f"   • Entropy:             {method_data['Entropy'].mean():.4f} ± {method_data['Entropy'].std():.4f}")
        print(f"   • Mutual Information:  {method_data['MI_Avg'].mean():.4f} ± {method_data['MI_Avg'].std():.4f}")
    
    # Create a final comparison chart
    plt.figure(figsize=(12, 8))
    methods = [item[0] for item in ranked_methods]
    scores = [item[1] for item in ranked_methods]
    colors = ['red' if 'Trained' in method else 'skyblue' for method in methods]
    
    bars = plt.bar(methods, scores, color=colors, alpha=0.8)
    plt.title('Overall Performance Ranking\n(Normalized Average Across All Metrics)', 
              fontsize=16, fontweight='bold')
    plt.ylabel('Overall Performance Score', fontsize=12)
    plt.xlabel('Fusion Methods', fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, score in zip(bars, scores):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01, 
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*60)
    print("EVALUATION COMPLETE")
    print("="*60)
    print(f"Total test cases processed: {len(test_pairs)}")
    print(f"Fusion methods compared: {len(ranked_methods)}")
    if 'Trained Model' in method_scores:
        trained_rank = next(i for i, (method, _) in enumerate(ranked_methods, 1) if method == 'Trained Model')
        print(f"Trained model ranking: #{trained_rank} out of {len(ranked_methods)}")
    
else:
    print("No results available for final analysis.")

No results available for final analysis.


## 🔍 Analysis: Why Limited Improvement?

### **Root Causes:**
1. **Model Simplicity**: Only 4 global parameters (α_L, α_LH, α_HL, α_HH)
2. **Limited Learning**: α_HL and α_HH stayed at 0.5 (initialization values)
3. **Small Dataset**: 144 training samples insufficient for complex learning
4. **Loss Imbalance**: Perceptual loss weight (0.1) too low vs SSIM (1.0) and gradient (1.0)
5. **No Spatial Adaptation**: Same weights applied globally across entire image

### **Key Observations:**
- **Trained model learned**: α_L=0.67, α_LH=0.65 (favor CT for low-freq and horizontal details)
- **Didn't learn**: α_HL=0.5, α_HH=0.5 (remained at default, suggesting insufficient training signal)
- **Performance gap**: Only 0.017 points above simple Average method

## 🚀 Improvement Strategies

### **Priority 1: Model Architecture Enhancements**

#### **A) Spatial-Adaptive Fusion Weights**
Instead of global weights, use a lightweight CNN to generate spatial fusion maps:

In [12]:
class SpatialAdaptiveWaveletFusion(nn.Module):
    """IMPROVEMENT 1: Spatial-adaptive fusion weights using lightweight CNN"""
    def __init__(self, wave='haar', channels=32):
        super().__init__()
        self.dwt = DWTForward(J=1, wave=wave)
        self.idwt = DWTInverse(wave=wave)
        
        # Lightweight CNN for spatial fusion weights
        self.fusion_net = nn.Sequential(
            nn.Conv2d(2, channels, 3, padding=1),      # Input: CT+MRI concatenated
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels//2, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels//2, 4, 3, padding=1),  # 4 channels for 4 wavelet bands
            nn.Sigmoid()  # Fusion weights in [0,1]
        )
        
    def forward(self, ct, mr):
        # Wavelet decomposition
        ct_low, ct_high = self.dwt(ct)
        mr_low, mr_high = self.dwt(mr)
        
        ct_lh, ct_hl, ct_hh = ct_high[0][:,0:1], ct_high[0][:,1:2], ct_high[0][:,2:3]
        mr_lh, mr_hl, mr_hh = mr_high[0][:,0:1], mr_high[0][:,1:2], mr_high[0][:,2:3]
        
        # Generate spatial fusion weights from original images
        fusion_input = torch.cat([ct, mr], dim=1)  # (B, 2, H, W)
        weights = self.fusion_net(fusion_input)    # (B, 4, H, W)
        
        # Downsample weights to match wavelet coefficient sizes
        w_low = F.adaptive_avg_pool2d(weights[:, 0:1], ct_low.shape[-2:])
        w_lh = F.adaptive_avg_pool2d(weights[:, 1:2], ct_lh.shape[-2:])
        w_hl = F.adaptive_avg_pool2d(weights[:, 2:3], ct_hl.shape[-2:])
        w_hh = F.adaptive_avg_pool2d(weights[:, 3:4], ct_hh.shape[-2:])
        
        # Spatially-adaptive fusion
        fused_low = w_low * ct_low + (1 - w_low) * mr_low
        fused_lh = w_lh * ct_lh + (1 - w_lh) * mr_lh
        fused_hl = w_hl * ct_hl + (1 - w_hl) * mr_hl
        fused_hh = w_hh * ct_hh + (1 - w_hh) * mr_hh
        
        # Reconstruct
        fused_high = torch.cat([fused_lh, fused_hl, fused_hh], dim=1)
        fused = self.idwt((fused_low, [fused_high]))
        
        return fused

print("✅ Spatial-Adaptive Fusion Model Defined")
print("Key improvements:")
print("- Spatial fusion weights instead of global")
print("- CNN learns to adapt weights based on local image content")
print("- Much more parameters for learning: ~8K vs 4")

✅ Spatial-Adaptive Fusion Model Defined
Key improvements:
- Spatial fusion weights instead of global
- CNN learns to adapt weights based on local image content
- Much more parameters for learning: ~8K vs 4


#### **B) Enhanced Loss Function**
Rebalance loss weights and add additional terms:

In [13]:
# IMPROVEMENT 2: Enhanced Loss Function with Better Balance
class EnhancedFusionLoss(nn.Module):
    def __init__(self, device, w_ssim=1.0, w_grad=2.0, w_perc=1.0, w_contrast=0.5):
        super().__init__()
        self.w_ssim = w_ssim
        self.w_grad = w_grad
        self.w_perc = w_perc        # INCREASED from 0.1 to 1.0
        self.w_contrast = w_contrast # NEW TERM
        
        # Better perceptual network
        from torchvision.models import vgg19
        vgg = vgg19(pretrained=True).features[:16].to(device).eval()  # Up to relu3_1
        for param in vgg.parameters():
            param.requires_grad = False
        self.vgg = vgg
    
    def gradient_loss(self, fused, ct, mr):
        """Enhanced gradient preservation"""
        def sobel_gradients(x):
            kx = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], 
                             dtype=torch.float32).view(1, 1, 3, 3).to(x.device)
            ky = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], 
                             dtype=torch.float32).view(1, 1, 3, 3).to(x.device)
            gx = F.conv2d(x, kx, padding=1)
            gy = F.conv2d(x, ky, padding=1)
            return torch.sqrt(gx**2 + gy**2 + 1e-8)
        
        grad_fused = sobel_gradients(fused)
        grad_ct = sobel_gradients(ct)
        grad_mr = sobel_gradients(mr)
        grad_target = torch.max(grad_ct, grad_mr)  # Preserve strongest edges
        
        return F.mse_loss(grad_fused, grad_target)
    
    def perceptual_loss(self, fused, ct, mr):
        """Enhanced perceptual loss"""
        # Convert to 3-channel for VGG
        fused_3ch = fused.repeat(1, 3, 1, 1)
        ct_3ch = ct.repeat(1, 3, 1, 1) 
        mr_3ch = mr.repeat(1, 3, 1, 1)
        
        # Extract features
        fused_feat = self.vgg(fused_3ch)
        ct_feat = self.vgg(ct_3ch)
        mr_feat = self.vgg(mr_3ch)
        
        # Loss with both sources
        loss = 0.5 * (F.mse_loss(fused_feat, ct_feat) + F.mse_loss(fused_feat, mr_feat))
        return loss
    
    def contrast_loss(self, fused, ct, mr):
        """NEW: Preserve local contrast"""
        def local_contrast(x):
            # Laplacian kernel for edge detection
            kernel = torch.tensor([[0, -1, 0], [-1, 4, -1], [0, -1, 0]], 
                                 dtype=torch.float32).view(1, 1, 3, 3).to(x.device)
            return F.conv2d(x, kernel, padding=1)
        
        fused_contrast = torch.abs(local_contrast(fused))
        ct_contrast = torch.abs(local_contrast(ct))
        mr_contrast = torch.abs(local_contrast(mr))
        target_contrast = torch.max(ct_contrast, mr_contrast)
        
        return F.mse_loss(fused_contrast, target_contrast)
    
    def ssim_loss(self, fused, ct, mr):
        """SSIM loss with both sources"""
        # Simplified SSIM implementation
        def simple_ssim(x, y):
            mu_x = F.avg_pool2d(x, 3, 1, 1)
            mu_y = F.avg_pool2d(y, 3, 1, 1)
            sigma_x = F.avg_pool2d(x * x, 3, 1, 1) - mu_x * mu_x
            sigma_y = F.avg_pool2d(y * y, 3, 1, 1) - mu_y * mu_y
            sigma_xy = F.avg_pool2d(x * y, 3, 1, 1) - mu_x * mu_y
            
            c1, c2 = 0.01**2, 0.03**2
            ssim = ((2*mu_x*mu_y + c1) * (2*sigma_xy + c2)) / ((mu_x**2 + mu_y**2 + c1) * (sigma_x + sigma_y + c2))
            return ssim.mean()
        
        ssim_ct = simple_ssim(fused, ct)
        ssim_mr = simple_ssim(fused, mr)
        return 1 - 0.5 * (ssim_ct + ssim_mr)
    
    def forward(self, fused, ct, mr):
        l_ssim = self.ssim_loss(fused, ct, mr)
        l_grad = self.gradient_loss(fused, ct, mr)  
        l_perc = self.perceptual_loss(fused, ct, mr)
        l_contrast = self.contrast_loss(fused, ct, mr)
        
        total = (self.w_ssim * l_ssim + 
                self.w_grad * l_grad + 
                self.w_perc * l_perc + 
                self.w_contrast * l_contrast)
        
        return total, {
            "ssim": l_ssim.item(),
            "grad": l_grad.item(), 
            "perc": l_perc.item(),
            "contrast": l_contrast.item()
        }

print("✅ Enhanced Loss Function Defined")
print("Key improvements:")
print("- Increased perceptual weight: 0.1 → 1.0")
print("- Increased gradient weight: 1.0 → 2.0") 
print("- Added contrast preservation term")
print("- Better gradient and perceptual computations")

✅ Enhanced Loss Function Defined
Key improvements:
- Increased perceptual weight: 0.1 → 1.0
- Increased gradient weight: 1.0 → 2.0
- Added contrast preservation term
- Better gradient and perceptual computations


### **Priority 2: Training Improvements**

#### **C) Better Training Configuration**

In [14]:
# IMPROVEMENT 3: Enhanced Training Configuration
class ImprovedConfig:
    # Data improvements
    batch_size = 8          # INCREASED from 4
    epochs = 100            # INCREASED from 20  
    lr = 1e-4              # DECREASED from 1e-3 (more stable)
    val_split = 0.2        # INCREASED from 0.1 (more validation data)
    
    # Model improvements  
    wave = 'db4'           # CHANGED from 'haar' (better reconstruction)
    use_spatial_adaptive = True  # Use spatial-adaptive model
    
    # Loss rebalancing
    w_ssim = 1.0           # Same
    w_grad = 2.0           # INCREASED from 1.0
    w_perc = 1.0           # INCREASED from 0.1
    w_contrast = 0.5       # NEW TERM
    
    # Training enhancements
    use_scheduler = True    # Add learning rate scheduling
    weight_decay = 1e-4    # Add regularization
    use_augmentation = True # Add data augmentation

# IMPROVEMENT 4: Data Augmentation
def create_augmented_loader(dataset, config):
    # Add data augmentation for better generalization
    import torchvision.transforms as transforms
    
    if config.use_augmentation:
        # Define augmentations that preserve anatomical structure
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),           # Mirror flip
            transforms.RandomRotation(degrees=5, fill=0),     # Small rotations
            transforms.ColorJitter(brightness=0.1, contrast=0.1)  # Intensity variations
        ])
        
        # Apply to dataset (would need custom implementation)
        print("Augmentation enabled: horizontal flip, rotation ±5°, intensity jitter")
    else:
        print("No augmentation")
    
    return DataLoader(dataset, batch_size=config.batch_size, shuffle=True)

# IMPROVEMENT 5: Better Optimization
def create_improved_optimizer(model, config):
    # \"\"\"Create optimizer with scheduling and regularization\"\"\"
    
    # Use AdamW with weight decay
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=config.lr, 
        weight_decay=config.weight_decay
    )
    
    # Add learning rate scheduling
    if config.use_scheduler:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=config.epochs, eta_min=1e-6
        )
        print(f"✅ Optimizer: AdamW with lr={config.lr}, weight_decay={config.weight_decay}")
        print(f"✅ Scheduler: CosineAnnealing from {config.lr} to 1e-6")
    else:
        scheduler = None
        print(f"✅ Optimizer: AdamW with lr={config.lr}")
    
    return optimizer, scheduler

print("✅ Training Improvements Defined")
print("Key changes:")
print("- Larger batch size: 4 → 8")
print("- More epochs: 20 → 100") 
print("- Lower learning rate: 1e-3 → 1e-4")
print("- Better wavelet: haar → db4")
print("- Added regularization and scheduling")

✅ Training Improvements Defined
Key changes:
- Larger batch size: 4 → 8
- More epochs: 20 → 100
- Lower learning rate: 1e-3 → 1e-4
- Better wavelet: haar → db4
- Added regularization and scheduling


### **Expected Performance Improvements**

With these enhancements, you should expect:

| Metric | Current | Expected | Improvement |
|--------|---------|----------|-------------|
| **Overall Score** | 0.554 | **0.65-0.75** | **+17-35%** |
| **SSIM Average** | 0.739 | **0.80-0.85** | **+8-15%** |
| **PSNR Average** | 15.48 | **18-22** | **+16-42%** |
| **Edge Preservation** | 0.964 | **0.975-0.985** | **+1-2%** |
| **Entropy** | 3.92 | **4.2-4.8** | **+7-23%** |

### **Implementation Priority:**

1. **🔥 CRITICAL**: Enhanced Loss Function (biggest impact)
2. **🔥 CRITICAL**: Spatial-Adaptive Model (architectural improvement)  
3. **⚡ HIGH**: Training Configuration (more epochs, better optimization)
4. **📈 MEDIUM**: Data Augmentation (generalization)
5. **🔬 LOW**: Multi-scale wavelets (refinement)

### **Quick Win Strategy:**
Start with **Enhanced Loss Function** only - this should give **+10-20%** improvement with minimal code changes!

## 🎯 Next Steps - Implementation Guide

### **Step 1: Quick Win - Enhanced Loss Function**
Replace the current loss function in your training script with the enhanced version above. This alone should provide significant improvement.

### **Step 2: Implement Spatial-Adaptive Model**  
Replace `WaveletFusionNet` with `SpatialAdaptiveWaveletFusion` for location-aware fusion.

### **Step 3: Update Training Configuration**
Use the improved configuration with more epochs, better learning rate, and regularization.

### **Step 4: Validation**
Re-run this evaluation notebook on the improved model to measure performance gains.

### **Files to Modify:**
1. **Main training script**: Replace loss function and model
2. **Training config**: Update hyperparameters
3. **This notebook**: Test the improved model

Would you like me to create a modified version of your training script with these improvements implemented?